In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

In [ ]:
from helper_evaluation import set_all_seeds, set_deterministic, compute_confusion_matrix
from helper_train import train_model
from helper_plotting import plot_training_loss, plot_accuracy, show_examples, plot_confusion_matrix
from helper_dataset import get_dataloaders_mnist

RANDOM_SEED: Seed for random number generators to ensure reproducibility.

BATCH_SIZE: Number of samples processed in one pass.

NUM_EPOCHS: Number of epochs for training.

DEVICE: Specifies whether to use GPU or CPU based on availability.

In [ ]:
RANDOM_SEED = 123
BATCH_SIZE = 256
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

set_all_seeds(RANDOM_SEED): Sets seeds for random number generators to ensure consistent results.

In [ ]:
set_all_seeds(RANDOM_SEED)
#set_deterministic()

resize_transform: Defines a sequence of transformations to be applied to images:

Resize: Resizes images to 32x32 pixels.

ToTensor: Converts images to tensors.

Normalize: Normalizes the pixel values to have a mean of 0.5 and a standard
deviation of 0.5.

In [ ]:
resize_transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize((32, 32)),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5,), (0.5,))])

get_dataloaders_mnist: Function to load the MNIST dataset and apply transformations. Returns dataloaders for training, validation, and testing.

In [ ]:
train_loader, valid_loader, test_loader = get_dataloaders_mnist(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=resize_transform,
    test_transforms=resize_transform)

Prints dimensions of a batch of images and labels, and shows class labels for the first 10 examples to verify the data.

In [ ]:
# Checking the dataset
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

In [ ]:
class LeNet5(torch.nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super().__init__()

        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = torch.nn.Sequential(

            torch.nn.Conv2d(in_channels, 6, kernel_size=5),
            torch.nn.Tanh(),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(6, 16, kernel_size=5),
            torch.nn.Tanh(),
            torch.nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(16*5*5, 120),
            torch.nn.Tanh(),
            torch.nn.Linear(120, 84),
            torch.nn.Tanh(),
            torch.nn.Linear(84, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        return logits

Creates an instance of LeNet5 for grayscale images and 10 classes, then moves the model to the specified device (GPU/CPU).

Optimizer: Uses Stochastic Gradient Descent (SGD) with a learning rate of 0.1.

Scheduler: Adjusts the learning rate based on the validation performance.

train_model: Function to train the model and return lists of loss and accuracy metrics for training and validation.

plot_training_loss: Plots training loss over epochs.

plot_accuracy: Plots accuracy for training and validation sets.



In [ ]:
model = LeNet5(grayscale=True,
               num_classes=10)

model = model.to(DEVICE)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=100)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([80, 100])
plt.show()

model.cpu(): Moves model to CPU for evaluation.

show_examples: Displays example predictions from the test set.

In [ ]:
model.cpu()
show_examples(model=model, data_loader=test_loader)

class_dict: Maps class indices to human-readable labels.

compute_confusion_matrix: Computes the confusion matrix based on model predictions.

plot_confusion_matrix: Plots the confusion matrix to visualize performance across different classes.

In [ ]:
class_dict = {0: '0',
              1: '1',
              2: '2',
              3: '3',
              4: '4',
              5: '5',
              6: '6',
              7: '7',
              8: '8',
              9: '9'}

mat = compute_confusion_matrix(model=model, data_loader=test_loader, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())
plt.show()